In [ ]:
# Importing modules
import pandas as pd
import os
import nltk
#nltk.download('stopwords')
#nltk.download ('wordnet')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
# Importing Gensim
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
import gensim.corpora as corpora


In [ ]:
#Download all papers as a zip file.  Extract them to docx in your folder, XXX.  Replace XXX with name of the folder.
#In the same folder, each docx file will be converted into a txt file.
import os
import docx2txt
import io

# Get the folder name
folder_name = "YOUR FOLDER NAME"

# Iterate over the folder entries
for entry in os.listdir(folder_name):

    # Check if the entry is a docx file
    if entry.endswith(".docx"):

        # Get the docx file content
        with open(os.path.join(folder_name, entry), "rb") as docx_file:
            docx_content = docx_file.read()

        # Convert the docx content to text
        with io.BytesIO(docx_content) as bytes_io:
            txt_content = docx2txt.process(bytes_io)

        # Write the text content to a file
        with open(os.path.join(folder_name, entry[:-5] + ".txt"), "w", encoding="utf-8") as txt_file:
            txt_file.write(txt_content)

print("All done!")

In [ ]:
# find all the txt files in the dataset folder.  Every XXX you should replace with your own folder/files
inputs = []
for file in os.listdir(folder_name):
    if file.endswith(".txt"):
        inputs.append(os.path.join(folder_name, file))
 
 
# concatanate all txt files in a file called merged_file.txt
with open('merged_file.txt', 'w', encoding="utf-8") as outfile:
    for fname in inputs:
        with open(fname, encoding="utf-8", errors='ignore') as infile:
            outfile.write(infile.read())

In [ ]:
#calls the merged file doc_complete
with open('merged_file.txt',encoding="utf-8") as f:
    doc_complete=f.readlines()

In [ ]:
#preprocesses the text.  The stop_words variable is extended based on the need from the files. Result is list of lists
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend (['could','overall','able','time','thing','way','made','something','course','would','think','life','semester','good','professor','satlow',
                    'also','well','really','may','xmln','iqr', 'ukɠh', 'rx', 'utgt', 'rkuj', 'gh', 'ce', 'hi', 'aιiq', 'kjh', 'xp', 'jv', 'mbc', 'idk', 'qk', 'goͼ', 'vf', 'gafқ', 'final', 'reflective', 'essay', 'vƃښl', 'os', 'pk', 'word', 'settings', 'xmln', 'uzx', 'uդ', 'fh', 'kr', 'dgjcwx', 'dsh', 'jhp', 'pk', 'word', 'numbering', 'xmlmnpk', 'pk', 'word', 'settings', 'xmln', 'uzx', 'uդ', 'fh', 'kr', 'dgjcwx', 'dsh', 'jhp', 'pk', 'ߤlz', 'content_types', 'xml', 'ub袪', 'væ', 'qu', "numbering","reflection","class",'rels','one','like','happiness','pursuit'])                
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = doc_complete
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:100])



In [ ]:
#eliminates all of the empty lists within the list
res = [ele for ele in data_words if ele != []]

In [ ]:
#useful step for seeing frequencies and adjusting stop_words
import pandas as pd
pd.Series(res).value_counts()

In [ ]:
data_words=res

In [ ]:
#Prepares for topic modeling
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

In [ ]:
#Creates a file, data_words, that can either be printed out, cut, and pasted into a Word Cloud generator or used below to generate a word cloud
res1 = [' '.join(ele) for ele in data_words]
' '.join(str(e) for e in res1)
string=' '.join([str(item) for item in res1])
text_file=open ("XXX","w", encoding="utf-8")
n=text_file.write(string)
text_file.close()

In [ ]:
#creates topic models.  You may need to experiment with the num_topics parameter
from pprint import pprint
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
#Creates the word cloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Get the text to create the word cloud from
text = data_words

# Convert the list of lists to a string
text = " ".join(" ".join(list) for list in text)

# Create the word cloud object
wordcloud = WordCloud(width=800, height=800, background_color="white")

# Generate the word cloud
wordcloud.generate(text)

# Plot the word cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
#Creates a jpg on your computer.  
wordcloud.to_file("wordcloud.jpg")